In [1]:
import os
from glob import glob 
import tensorflow as tf

C:\Users\junhoning\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
master = ''  # Name of the TensorFlow master to use.
task = 0  # task id
num_clones = 1  # Number of clones to deploy per worker.
clone_on_cpu = False  # Force clones to be deployed on CPU.  Note that even if 
                      # set to False (allowing ops to run on gpu), some ops may
                      # still be run on the CPU if they have no GPU kernel.
worker_replicas = 1  # Number of worker+trainer replicas.
ps_tasks = 0  # Number of parameter server tasks. If None, does not use a parameter server.
train_dir = ''  # Directory to save the checkpoints and training summaries.
pipeline_config_path = ''  # Directory to save the checkpoints and training summaries.
train_config_path = ''  # Path to a pipeline_pb2.TrainEvalPipelineConfig config file. If provided, other configs are ignored
input_config_path = ''  # Path to a train_pb2.TrainConfig config file.
model_config_path = ''  # Path to a model_pb2.DetectionModel config file.

In [ ]:
assert train_dir, '`train_dir` is missing.'
if task == 0: tf.gfile.MakeDirs(train_dir)
if pipeline_config_path:
    configs = config_util.get_configs_from_pipeline_file(
        pipeline_config_path)
    if task == 0:
        tf.gfile.Copy(FLAGS.pipeline_config_path,
                      os.path.join(FLAGS.train_dir, 'pipeline.config'),
                      overwrite=True)
else:
    configs = config_util.get_configs_from_multiple_files(
        model_config_path=model_config_path,
        train_config_path=train_config_path,
        train_input_config_path=input_config_path)
    if FLAGS.task == 0:
        for name, config in [('model.config', model_config_path),
                             ('train.config', train_config_path),
                             ('input.config', input_config_path)]:
            tf.gfile.Copy(config, os.path.join(train_dir, name),
                          overwrite=True)

model_config = configs['model']
train_config = configs['train_config']
input_config = configs['train_input_config']